## AGUATHON ITA

#### Resumen 7

Se usan las conclusiones del notebook 6 para mejorar R2 con la suma de los cambios individuales

In [1]:
import os
import scipy
import numpy as np
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

## Fun

In [2]:
def load_rios():
    ''' 
    Load CSV using Pandas saved in current working directory
    '''
    cwd = os.getcwd()
    in_path = os.path.join(cwd,'data','ENTRADA')
    in_file = 'datos.csv'
    filename  = os.path.join(in_path,in_file)
    data = pd.read_csv(filename, parse_dates = ['time'], index_col = 'time') #, names=names)
    return data

def load_meteo(csvfile, cols):
    ''' 
    Load CSV using Pandas saved in current working directory
    '''
    cwd = os.getcwd()
    in_path = os.path.join(cwd, 'data', 'DatosPorEstacion')
    in_file = csvfile + '.csv'
    filename  = os.path.join(in_path,in_file)
    data = pd.read_csv(filename,sep=";", usecols = cols,
                       parse_dates = ['FECHA'], index_col='FECHA')
    return data


def column_shift_bak(df, dup_cols, N, delta ): 
    '''
    Deriving new features by shifting columns by a given timedelta. That is, each time series 
    is shifted back N*delta, N*2*delta, N*3*delta, where N is number of columns and delta
    time shift periods.
    '''    
    dfs = df.copy()
    for col in dup_cols:
        for i in range (1,N+1):
            colname = col + "_" + str(i*delta)
            dfs[colname] = dfs[col].shift(periods = i*delta)
    return dfs 

def column_shift(df, dup_cols, N, delta ): 
    '''
    Deriving new features by shifting columns by a given timedelta. That is, each time series 
    is shifted back N*delta, N*2*delta, N*3*delta, where N is number of columns and delta
    time shift periods.
    '''    
    for col in dup_cols:
        for i in range (1,N+1):
            colname = col + "_" + str(i*delta)
            df[colname] = df[col].shift(periods = i*delta)
    return df

In [17]:
############################################################## 
##     DATA LOADING
##############################################################

### RIOS
df_rios = load_rios()
df_rios.columns = ['Ala','Gri','Nov','Tau','Tud','Zar','Risk','P24','P48','P72']
start = df_rios.index[0]
end = df_rios.index[-1]

### Estaciones
estaciones = {'pa1':'9262-19530901-20190131',   # pna
              'pa2':'9263D-19750101-20190302',  # pna Aerop
              'za1':'9434-19410101-20190302',   # zar Aerop
              'za2':'9244X-19920204-20190302',  # zar Sos rey 
              'hu1':'9208E-20060201-20190302',  # huesca aragues
              'hu2':'9201K-19920101-20190302',  # huesca jaca
              'log':'9170-19481101-20190302',   # Logroño
              'cal':'9394X-19930401-20190302',  # Calatayud
             }



### TEMP
cols = ['FECHA','TMEDIA']
locations = ['pa1', 'pa2', 'za1', 'za2', 'log', 'cal', 'hu1', 'hu2']
dframes = []
for k,v in estaciones.items():
    if k in locations:
        df = load_meteo(v, cols)
        df = df.resample('H').ffill()
        df = df[start:end]
        df.columns = ['Tm_'+k]
        dframes.append(df)
df_temp = pd.concat(dframes, axis=1)




### RAIN
cols = ['FECHA','PRECIPITACION']
locations = ['pa1', 'pa2', 'hu1', 'hu2', 'log'] # mejores para lluvia
dframes = []
for k,v in estaciones.items():
    if k in locations:    
        df = load_meteo(v, cols)
        df = df.resample('H').ffill()
        df = df[start:end]
        df.columns = ['rain_'+ k] 
        dframes.append(df)
df_rain = pd.concat(dframes, axis=1)



    
### PRESION
locations = ['pa2', 'za1', 'log'] # solo estas tienen datos de presion
cols = ['FECHA','PRESMAX', 'PRESMIN']
dframes = []
for k,v in estaciones.items():
    # load selected locations only 
    if k in locations:
        #  Read csv into a DataFrame: df
        df = load_meteo(v, cols)
        df = df.resample('H').ffill()
        df = df[start:end]        
        df.columns = ['Pmax_'+ k, 'Pmin_'+ k] 
        dframes.append(df)
df_pres = pd.concat(dframes, axis=1)



####################################################################### 
######     DATA PREPARATION
######################################################################


#### RAIN DATA
df_rain.fillna(value = 0,inplace=True) 
# replace string values defined by AEMET
df_rain.replace(to_replace =['Ip','Acum','Varias'] , value =0 , inplace = True)
#convert Text columns to numeric
for col in df_rain.columns:
    df_rain[col] = pd.to_numeric(df_rain[col], errors='coerce')


#### TEMP DATA
df_temp.fillna(method = 'ffill',inplace=True) 
df_temp.fillna(method = 'bfill',inplace=True)

#### PRES DATA
df_pres.fillna(method = 'ffill',inplace=True) 
df_pres.fillna(method = 'bfill',inplace=True)

### SELECT RIOS DATA
target = 'P72'
# selected features
cols = ['Ala','Tud','Nov','Zar']#,'Gri']
cols.append(target)
df_rios = df_rios[cols]

### RIOS + LLUVIA + PRES (TEMP eliminada porque no ayuda)
#df = pd.concat([df_rios, df_rain, df_pres], axis=1)
#df = df_rios
df = pd.concat([df_rios, df_rain], axis=1)

 
N = 1       # number of derived columns per feature
delta = 8   # time delta in hr
dup_cols = ['Tud', 'Nov']#, 'Tud', 'Zar']
df = column_shift(df, dup_cols, N, delta)
    
    
N = 1       # number of derived columns per feature
delta = 145   # time delta in hr
dup_cols = ['Ala', 'Zar']#, 'Tud', 'Zar']
df = column_shift(df, dup_cols, N, delta)    
    
    
    
df.dropna(axis=0, how='any', inplace=True)


            
df.head()

,Ala,Tud,Nov,Zar,P72,rain_pa1,rain_pa2,rain_hu1,rain_hu2,rain_log,Tud_8,Nov_8,Ala_145,Zar_145
2008-01-07 01:00:00,0.9000,0.8425,1.8500,0.8300,0.8600,0.0,0.2,0.1,0.4,0.0,0.8400,1.8275,0.8100,0.74
2008-01-07 02:00:00,0.9025,0.8450,1.8500,0.8300,0.8600,0.0,0.2,0.1,0.4,0.0,0.8325,1.8175,0.8100,0.74
2008-01-07 03:00:00,0.8925,0.8500,1.8500,0.8200,0.8600,0.0,0.2,0.1,0.4,0.0,0.8300,1.8050,0.8100,0.74
2008-01-07 04:00:00,0.8900,0.8500,1.8575,0.8200,0.8600,0.0,0.2,0.1,0.4,0.0,0.8300,1.8025,0.8075,0.74
2008-01-07 05:00:00,0.8900,0.8400,1.8600,0.8175,0.8575,0.0,0.2,0.1,0.4,0.0,0.8300,1.8200,0.8000,0.74


In [18]:
############################################################### 
########       ML SPLIT AND REGRESSION
###############################################################


features = [x for x in df.columns if x != target]
Y = df[target]
X = df.loc[:,features]


kfold = KFold(n_splits=10, random_state=0)
model = LinearRegression()
scoring = 'r2'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
mn = results.mean()*100.0
sd = results.std()*100.0
print("N:{}, dt:{}, Accuracy: {:.3f}% ({:.3f}%)".format(N, delta, mn, sd))

N:1, dt:145, Accuracy: 83.739% (7.515%)


### Estudio Sensibilidad Variables
```
Resultados de estudio de notebook V5

Rios solo
* 66.601% (15.258%) Ala
* 70.748% (13.848%) Ala,Tud
* 70.570% (14.261%) Ala,Tud,Zar
* 70.027% (14.635%) Ala,Tud,Gri
* 69.781% (15.112%) Ala,Tud,Gri,Zar
* 71.320% (14.805%) Ala,Tud,Nov        <= (2)
* 71.393% (14.716%) Ala,Tud,Nov,Zar    <= (1)
* 71.279% (14.391%) Ala,Tud,Nov,Zar,Gri
* combinacion 1 la de mejor R2

Rios + Lluvia por estacion
* 80.646%  (9.153%) 1 + todas estaciones
* 80.002%  (9.534%) 1 + pa1
* 78.740% (10.846%) 1 + pa2
* 73.261% (13.912%) 1 + za1
* 73.850% (14.069%) 1 + za2
* 74.850% (13.424%) 1 + log
* 72.488% (13.854%) 1 + cal
* 76.010% (11.421%) 1 + hu1
* 74.148% (13.295%) 1 + hu2
* 80.805% (8.939%)  1 + pa1 + pa2 + hu1 + hu2
* 80.744% (9.111%)  1 + pa1 + pa2 + hu1 + hu2 + log
* Esta ultima es la mejor combinacion. Referencia para posteriores estudios

Rios + Temp
* 71.781% (15.308%) 1 + all locations
* 72.175% (14.747%) 1 + pa1
* 72.171% (14.655%) 1 + pa2
* 72.128% (14.525%) 1 + za1
* 71.894% (15.165%) 1 + za2
* 72.197% (14.415%) 1 + log
* 71.738% (14.184%) 1 + cal
* 72.630% (14.121%) 1 + hu1
* 72.156% (14.855%) 1 + hu2
* temp no tiene mucha influencia en R2

Rios + Pres (max & min)
* 74.339% (12.836%) 1 + pa2
* 73.991% (13.042%) 1 + zag1
* 73.870% (13.228%) 1 + log
* 76.010% (11.205%) 1 + pa2, zag1, log 
* pendiente estudiar solo Pmax, Pmin o diferencias


Resultados de V6 Duplicado de columna desfasadas (delta en hr) en el tiempo:

* Ala, delta = 144, Max = 81.443% (8.385%)
* Tud, delta =   8, Max = 83.475% (8.052%)
* Nov, delta =  10, MAx = 81.608% (8.508%)
* Zar, delta = 145, MAx = 81.952% (8.353%)

Resultados de V7

* Duplicando cols(Tud @ delta=8)
* Accuracy: 83.476% (8.052%)

* Duplicando cols(Tud @ delta= 8) + (Zar,Ala @ delta = 145)
* Accuracy: 83.712% (7.586%)

* Duplicando cols(Tud,Nov @ delta= 8) + (Zar,Ala @ delta = 145)
* Accuracy: 83.739% (7.515%)


```